# IEEE Challenge

In [102]:
import pandas as pd
import numpy as np
from src import paths
from src.read_data import DataReader
from src.data_preprocessor import DataPreprocessor
from src.utils.util_functions import key_with_max_value, ordered_values

ImportError: cannot import name 'ordered_values' from 'src.utils.util_functions' (C:\Users\janni\Dropbox\projects\2024_05 IEEE Suicide prevention\ieee-2024-suicide-prevention\src\utils\util_functions.py)

In [2]:
labelled = DataReader.get_posts_with_labels()

In [3]:
labelled.head()

,post,post_risk
0,Just kill me. Please! Just end my life! I beg ...,ideation
1,"There is no today, nor tommorrow. I dont have ...",behavior
2,Life so full of contradictions that it's not w...,indicator
3,I think I'm going to kill myself soon. I don't...,behavior
4,whats the point of living. no really. is there...,ideation


In [4]:
labelled['post_risk'].value_counts()

post_risk
ideation     190
behavior     140
indicator    129
attempt       41
Name: count, dtype: int64

In [5]:
labelled = DataPreprocessor.preprocess(labelled)
labelled.head()

2024-05-31 17:57:25,436 - INFO: Preprocesssing data
2024-05-31 17:57:25,441 - INFO: Removing HTML, non ASCI etc
2024-05-31 17:57:25,543 - INFO: Counting words


,post,post_risk,word_count
0,Just kill me. Please! Just end my life! I beg ...,ideation,14
1,"There is no today, nor tommorrow. I dont have ...",behavior,282
2,Life so full of contradictions that it's not w...,indicator,311
3,I think I'm going to kill myself soon. I don't...,behavior,54
4,whats the point of living. no really. is there...,ideation,142


In [6]:
labelled.groupby('post_risk')['word_count'].describe()

,count,mean,std,min,25%,50%,75%,max
post_risk,,,,,,,,
attempt,41.0,338.341463,427.035046,11.0,55.0,179.0,447.00,2095.0
behavior,140.0,177.314286,186.416841,2.0,55.0,94.0,230.25,830.0
ideation,190.0,161.473684,170.525415,6.0,54.0,98.5,214.00,998.0
indicator,129.0,135.333333,144.509047,1.0,44.0,90.0,188.00,1120.0


In [7]:
file_name = "data"
DataPreprocessor.to_json(labelled, f'{file_name}.jsonl')

In [8]:
!openai tools fine_tunes.prepare_data -f data.jsonl -q

Analyzing...

- Your file contains 500 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 1 examples that are very long. These are rows: [481]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start with a whitespace ch

In [11]:
from src.open_ai import client
train_file = client.files.create(file=open(f"{file_name}_prepared_train.jsonl", "rb"), purpose="fine-tune")
valid_file = client.files.create(file=open(f"{file_name}_prepared_valid.jsonl", "rb"), purpose="fine-tune")

fine_tuning_job = client.fine_tuning.jobs.create(training_file=train_file.id, validation_file=valid_file.id, model="babbage-002")

print(fine_tuning_job)

FineTuningJob(id='ftjob-26SCSYKJlwPoNupKn5QvasUA', created_at=1717176751, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-FM8nE4u0PEcj6crKCzO0dFT7', result_files=[], seed=1861505804, status='validating_files', trained_tokens=None, training_file='file-TUqg9LrTCqCIZqRg6fMCrBrX', validation_file='file-EGKmilIo2fKHALFsg7UGFRoE', estimated_finish=None, integrations=[], user_provided_suffix=None)


In [15]:
print(fine_tuning_job)

FineTuningJob(id='ftjob-26SCSYKJlwPoNupKn5QvasUA', created_at=1717176751, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-FM8nE4u0PEcj6crKCzO0dFT7', result_files=[], seed=1861505804, status='validating_files', trained_tokens=None, training_file='file-TUqg9LrTCqCIZqRg6fMCrBrX', validation_file='file-EGKmilIo2fKHALFsg7UGFRoE', estimated_finish=None, integrations=[], user_provided_suffix=None)


In [18]:
test = DataReader.get_test_set()
test.head()

,post
0,My mind is hell. After my breakup 6 months ago...
1,What's the point?. It's been 548 days since my...
2,I'm scared therapy wont work. My parents reali...
3,so bye. love you bitches. only a few hours lef...
4,Relapsed after 6 months. I don't think I'll ev...


In [114]:
fine_tuning_job.id

'ftjob-26SCSYKJlwPoNupKn5QvasUA'

In [28]:
# last job id 'ftjob-26SCSYKJlwPoNupKn5QvasUA'.
fine_tune_results = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
print(fine_tune_results.finished_at)

1717177168


In [54]:
ft_model = fine_tune_results.fine_tuned_model

# note that this calls the legacy completions api - https://platform.openai.com/docs/api-reference/completions
res = client.completions.create(model=ft_model, prompt=test['post'][15] + ' ->', max_tokens=1, temperature=0,logprobs=10)

In [78]:
result = [client.completions.create(model=ft_model, prompt=ex + ' ->', max_tokens=1, temperature=0,logprobs=10) for ex in test['post']]

In [62]:
res.choices[0].logprobs.tokens

[' behavior']

In [97]:
def unpack(response):
    log_probs = response.choices[0].logprobs.top_logprobs[0]
    prob ={k: np.exp(log_p)/np.sum(np.exp(list(log_probs.values()))) for k, log_p in log_probs.items()}
    out = {'ideation': 0, 'behavior': 0, 'indicator':0, 'attempt': 0}
    for token, p in prob.items():
        if token.startswith(" be"):
            out['behavior'] = out['behavior']+ prob[token]
        elif token.startswith((" id", "at")):
            out['ideation'] = out['ideation']+ prob[token]
        elif token.startswith(" at"):
            out['attempt'] = out['attempt']+ prob[token]
        elif token.startswith(" in"):
            out['indicator'] = out['indicator']+ prob[token]
    return out

In [98]:
probs = [unpack(res) for res in result]

In [99]:
probs[:2]

[{'ideation': 0.41600144062232575,
  'behavior': 0.35565867271386614,
  'indicator': 0.20832621373504148,
  'attempt': 0.02001367292876656},
 {'ideation': 0.49733861727356055,
  'behavior': 0.3016117459746736,
  'indicator': 0.18352530885777973,
  'attempt': 0.017524327893986006}]

In [108]:
test['suicide risk'] = [key_with_max_value(elem) for elem in probs]
test['probability distribution'] = [ordered_values(elem) for elem in probs]
test

,post,suicide risk,probability distribition,probability distribution
0,My mind is hell. After my breakup 6 months ago...,ideation,"[0.20833, 0.416, 0.35566, 0.02001]","[0.20833, 0.416, 0.35566, 0.02001]"
1,What's the point?. It's been 548 days since my...,ideation,"[0.18353, 0.49734, 0.30161, 0.01752]","[0.18353, 0.49734, 0.30161, 0.01752]"
2,I'm scared therapy wont work. My parents reali...,ideation,"[0.16726, 0.53765, 0.27948, 0.0156]","[0.16726, 0.53765, 0.27948, 0.0156]"
3,so bye. love you bitches. only a few hours lef...,behavior,"[0.32152, 0.20119, 0.45864, 0.01864]","[0.32152, 0.20119, 0.45864, 0.01864]"
4,Relapsed after 6 months. I don't think I'll ev...,ideation,"[0.18313, 0.47322, 0.32552, 0.01813]","[0.18313, 0.47322, 0.32552, 0.01813]"
...,...,...,...,...
95,"""Help me die. Hello, I'm sorry, I'm going to t...",ideation,"[0.18425, 0.47515, 0.325, 0.01559]","[0.18425, 0.47515, 0.325, 0.01559]"
96,"I'm still here, but I had enough of this. Hey ...",ideation,"[0.12563, 0.6057, 0.2529, 0.01577]","[0.12563, 0.6057, 0.2529, 0.01577]"
97,having a little bit of money helps but i'm sti...,ideation,"[0.17787, 0.47053, 0.33485, 0.01675]","[0.17787, 0.47053, 0.33485, 0.01675]"
98,I need therapy more than anything else but I h...,ideation,"[0.02243, 0.90336, 0.06846, 0.00575]","[0.02243, 0.90336, 0.06846, 0.00575]"


In [113]:
test[['suicide risk', 'probability distribution']].to_excel("Calculators.xlsx", index=True, index_label="index")